In [435]:
import glob
# This file should run basically insta fast. Bad coding for sure 
genSavePath = "F:/PythonProjects/NietoExcercise-1/SavedResults/"
day = "07" # Which month and days result you want to visualize in table
month = 11
repetitionValue = 35
repetitionName =  "udiplotAda3" #  "udrlBC30comma005ada"  #  "udrlBC4CVTest" # 
# 
repetitionString = f"{repetitionValue}{repetitionName}"
curSavePath = f"{genSavePath}{day}-{month}-{repetitionString}/"
savedResults = glob.glob(curSavePath + "savedBestSeed*")

skipSubjects = []
onlySignificant = True
minAcc = 0.2
print(len(savedResults))
print(len(savedResults)/9)

78
8.666666666666666


In [436]:
import pandas as pd
from array import ArrayType
from requests import delete
from tabulate import tabulate
from prettytable import PrettyTable as pt
import numpy as np


nameDict = dict()
nrOfNames = 0
kernelDict = dict()
nrOfKernel = 0
nrOfCombination = 0
currSeed = 0
currSubj = 0
combinationList = []
combinationResult = []


results = []
for path in savedResults:
    results.append(np.load(path, allow_pickle=True))

subjectAmount = 10
for x in range(subjectAmount):
    combinationResult.append([])

nrOfSeedsPerSubject = np.zeros(subjectAmount)
for s, subject in enumerate(results):
    currSubj = 0
    currSeed = 0
    subjectBestArray = np.empty([len(subject)], dtype=object)

    currSeed = subject[0]
    currSubj = subject[1]

    if currSubj in skipSubjects:
        continue
    nrOfSeedsPerSubject[currSubj] += 1

    for aResults in subject[2]:
        for aResults2 in aResults:
            # If the np array comes from svm results
            if len(aResults2) < 6:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]
                if ares > 0.98:
                    ares = 0.99
                akernel = aResults2[2]
                if "lin" in akernel:
                    continue
                if "sig" in akernel:
                    continue
                aC = round(aResults2[3], 2)

                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [nameDict[aName], kernelDict[akernel], aC] not in combinationList:
                    combinationList.append([nameDict[aName], kernelDict[akernel], aC])
                    nrOfCombination += 1

                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aC]
                )
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])
                    
                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][
                        index
                    ][1] * (1 - (1 / nrOfSeedsPerSubject[currSubj])) + ares * (
                        1 / nrOfSeedsPerSubject[currSubj]
                    )

                else:

                    combinationResult[currSubj].append([combNr, ares])

                    # If the np array comes from NN results
            else:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]
                akernel = aResults2[2]
                aLz = round(aResults2[3], 2)
                areg = round(aResults2[4], 2)
                adrp = round(aResults2[5], 2)
                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [
                    nameDict[aName],
                    kernelDict[akernel],
                    aLz,
                    areg,
                    adrp,
                ] not in combinationList:
                    combinationList.append(
                        [nameDict[aName], kernelDict[akernel], aLz, areg, adrp]
                    )
                    nrOfCombination += 1
                # combResult = []
                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aLz, areg, adrp]
                )
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])
                  
                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][
                        index
                    ][1] * (1 - (1 / nrOfSeedsPerSubject[currSubj])) + ares * (
                        1 / nrOfSeedsPerSubject[currSubj]
                    )

                else:

                    combinationResult[currSubj].append([combNr, ares])


# Making table

# headers = [f"{x[0]}:{x[1]}:{x[2]}:" for x in combinationList]
headers = np.arange(nrOfCombination)
subjectResultArray = np.empty([10, nrOfCombination], dtype=object)
subjectNrList = []

comboAverage = np.zeros([1, nrOfCombination])
comboTotal = np.zeros([1, nrOfCombination])
nrOfSubjects = np.zeros(nrOfCombination)

topTen = []
topThreeAllSubjects = []
lowestTopRes = [0, 0]
for subNr, cResult in enumerate(combinationResult):
    topThreeThisSubject = []
    resArray = np.zeros([nrOfCombination])
    if len(cResult) != 0:
        for res in cResult:
            if onlySignificant == True:
                if res[1] < minAcc:
                    continue
            comboAverage[0, res[0]] = comboAverage[0, res[0]] + res[1]
            comboTotal[0, res[0]] = comboTotal[0, res[0]] + res[1]
            nrOfSubjects[res[0]] += 1
            subjectResultArray[subNr, res[0]] = res[1]

            if len(topTen) < 10 or topTen[0][0] < res[1]:

                if len(topTen) == 10:
                    topTen.reverse()
                    topTen.pop()

                dataF = list(nameDict.keys())[combinationList[res[0]][0]]
                kernel = list(kernelDict.keys())[combinationList[res[0]][1]]
                cVal = combinationList[res[0]][2]
                topTen.append([res[1], subNr, dataF, kernel, cVal])
                topTen.sort()

            
            
            if len(topThreeThisSubject) < 2 or topThreeThisSubject[0][0] < res[1]:

                if len(topThreeThisSubject) == 2:
                    topThreeThisSubject.reverse()
                    topThreeThisSubject.pop()

                dataF = list(nameDict.keys())[combinationList[res[0]][0]]
                kernel = list(kernelDict.keys())[combinationList[res[0]][1]]
                cVal = combinationList[res[0]][2]
                topThreeThisSubject.append([res[1], subNr, dataF, kernel, cVal])
                topThreeThisSubject.sort()
    if len(topThreeThisSubject)>0:
        topThreeAllSubjects.append(topThreeThisSubject)
# Print the table!
comboAverage = comboAverage / nrOfSubjects
print(comboAverage.shape)
print(subjectResultArray.shape)
pd.set_option("display.max_columns", None)

# For loop through zip of headers and columns,
# If column has all values == None, Remove column and header.
subjectResultArray = np.array(subjectResultArray)
emptyCols = []
atleastOneEmpty = False
for hcol, in zip(headers):
    if all(subjectResultArray[:,hcol] == None):
        atleastOneEmpty = True
        emptyCols.append(hcol)
if atleastOneEmpty: 
    subjectResultArray = np.delete(subjectResultArray, np.array(emptyCols),1)
    headers = np.delete(headers, np.array(emptyCols),0)
    comboAverage = np.delete(comboAverage, np.array(emptyCols),1)
    comboTotal = np.delete(comboTotal, np.array(emptyCols),1)
    nrOfSubjects = np.delete(nrOfSubjects, np.array(emptyCols),0)


table = pd.DataFrame(subjectResultArray, columns=headers)
display(table)




(1, 108)
(10, 108)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,0.908889,0.871111,0.897778,0.903333,0.871111,0.822222,0.822222,0.848889,0.844444,0.444444,0.858889,0.86,0.843333,0.816667,0.777778,0.883333,0.877778,0.894444,0.904444,0.644444,0.844444,0.838889,0.855556,0.855556,0.561111,0.914444,0.876667,0.897778,0.903333,0.683333,0.766667,0.894444,0.854444,0.911111,0.882222,0.887778,0.848889,0.816667,0.838889,0.865556,0.898889,0.914444,0.827778,0.838889,0.876667,0.872222,0.666667,0.677778,0.683333,0.738889,0.738889,0.738889,0.883333,0.905556,0.894444,0.844444,0.887778,0.875556,0.855556,0.898889,0.887778,0.893333,0.91,0.893333,0.875556,0.881111,0.892222,0.86,0.871111,0.827778,0.794444,0.822222,0.788889,0.816667,0.844444,0.844444,0.882222,0.887778,0.805556,0.838889,0.877778,0.871111,0.882222,0.788889,0.893333,0.9,0.888889,0.794444,0.875556,0.872222,0.894444,0.871111,0.511111,0.661111,0.616667,0.466667,0.5,0.572222,0.583333,0.788889,0.6,0.544444,0.838889,0.533333,0.561111,0.494444,0.433333,0.577778
2,0.851852,0.865741,0.892407,0.893519,0.887778,0.694444,0.703704,0.712963,0.689815,0.421296,0.763889,0.694444,0.740741,0.708333,0.787037,0.662037,0.615741,0.62963,0.62037,0.458333,0.652778,0.648148,0.662037,0.657407,0.481481,0.865741,0.87037,0.906296,0.916667,0.490741,0.518519,0.795185,0.782407,0.814815,0.787037,0.625,0.722222,0.578704,0.652778,0.800926,0.796296,0.833333,0.615741,0.782407,0.791667,0.75463,0.467593,0.5,0.5,0.50463,0.490741,0.481481,0.74537,0.763889,0.740741,0.763889,0.805556,0.791667,0.819444,0.878519,0.832222,0.800926,0.800926,0.777778,0.62963,0.675926,0.652778,0.731481,0.712963,0.722222,0.513889,0.509259,0.490741,0.652778,0.648148,0.657407,0.824074,0.847222,0.560185,0.759259,0.787037,0.763889,0.819444,0.537037,0.805556,0.805556,0.722222,0.537037,0.796296,0.828704,0.740741,0.891296,0.439815,0.564815,0.472222,0.453704,0.416667,0.430556,0.449074,0.787037,0.435185,0.449074,0.875,0.486111,0.425926,0.449074,0.407407,0.425926
3,0.592593,0.567901,0.592593,0.648148,0.771605,0.530864,0.512346,0.487654,0.537037,0.444444,0.481481,0.438272,0.518519,0.54321,0.592593,0.469136,0.41358,0.487654,0.481481,0.469136,0.512346,0.493827,0.475309,0.481481,0.432099,0.623457,0.549383,0.604938,0.697531,0.462963,0.432099,0.62963,0.524691,0.580247,0.58642,0.5,0.54321,0.518519,0.518519,0.5,0.549383,0.679012,0.518519,0.524691,0.518519,0.5,0.425926,0.450617,0.41358,0.462963,0.41358,0.425926,0.530864,0.54321,0.580247,0.469136,0.518519,0.493827,0.5,0.537037,0.611111,0.530864,0.54321,0.58642,0.438272,0.530864,0.475309,0.487654,0.567901,0.487654,0.487654,0.512346,0.506173,0.506173,0.469136,0.487654,0.493827,0.574074,0.5,0.487654,0.475309,0.469136,0.54321,0.506173,0.512346,0.518519,0.54321,0.530864,0.5,0.54321,0.530864,0.722222,0.419753,0.493827,0.5,0.388889,0.37037,0.407407,0.45679,0.518519,0.462963,0.425926,0.660494,0.475309,0.462963,0.376543,0.37037,0.419753
4,0.902778,0.87037,0.922593,0.948148,0.94463,0.878519,0.842593,0.865741,0.865741,0.50463,0.842593,0.875,0.856481,0.837963,0.842593,0.893519,0.833333,0.884259,0.87963,0.486111,0.819444,0.787037,0.814815,0.805556,0.518519,0.912037,0.915556,0.922593,0.948148,0.717593,0.810185,0.945741,0.805556,0.907407,0.915556,0.87

In [437]:
x = 0

pd.set_option("display.max_columns", None)
print()
nameDictNrs = np.ones([1, len(nameDict)])
nameDictNrs[0,:] = np.arange(len(nameDict)) * 3
print(nameDictNrs.shape)
table = pd.DataFrame(nameDictNrs , columns=nameDict.keys())
display(table)
# for names in nameDict.keys():
#     print(names, end="        ")
#     x +=1
#     #print(x)


(1, 108)


,dataHRCV-BC--dataFFTCV-BC,dataHRCV-BCcn3--dataFFTCV-BC,fftDatacn3BC--dataFFTCV-BC,welchDatacn3BC--dataFFTCV-BC,welchDatacn3BC--dataWCV-BC,dataHRCV-BC-dataFFTCV-BC--fftData,dataHRCV-BCcn3-dataFFTCV-BC--fftData,fftDatacn3BC-dataFFTCV-BC--fftData,welchDatacn3BC-dataFFTCV-BC--fftData,welchDatacn3BC-dataWCV-BC--fftData,dataHRCV-BC-dataFFTCV-BC--welchData,dataHRCV-BCcn3-dataFFTCV-BC--welchData,fftDatacn3BC-dataFFTCV-BC--welchData,welchDatacn3BC-dataFFTCV-BC--welchData,welchDatacn3BC-dataWCV-BC--welchData,dataHRCV-BC-dataFFTCV-BC--dataHR,dataHRCV-BCcn3-dataFFTCV-BC--dataHR,fftDatacn3BC-dataFFTCV-BC--dataHR,welchDatacn3BC-dataFFTCV-BC--dataHR,welchDatacn3BC-dataWCV-BC--dataHR,dataHRCV-BC-dataFFTCV-BC--dataCorr1d,dataHRCV-BCcn3-dataFFTCV-BC--dataCorr1d,fftDatacn3BC-dataFFTCV-BC--dataCorr1d,welchDatacn3BC-dataFFTCV-BC--dataCorr1d,welchDatacn3BC-dataWCV-BC--dataCorr1d,dataHRCV-BC-dataWCV-BC--dataFFTCV-BC,dataHRCV-BCcn3-dataWCV-BC--dataFFTCV-BC,fftDatacn3BC-dataWCV-BC--dataFFTCV-BC,welchDatacn3BC-dataWCV-BC--dataFFTCV-BC,fftDataBC-dataHRCV-BC--dataFFTCV-BC,welchDataBC-dataHRCV-BC--dataFFTCV-BC,dataHRBC-dataHRCV-BC--dataFFTCV-BC,gaussianData-dataHRCV-BC--dataFFTCV-BC,gaussianDataBC-dataHRCV-BC--dataFFTCV-BC,dataCorr1dBC-dataHRCV-BC--dataFFTCV-BC,fftDatacn3-dataHRCV-BC--dataFFTCV-BC,welchDatacn3-dataHRCV-BC--dataFFTCV-BC,dataHRcn3-dataHRCV-BC--dataFFTCV-BC,dataCorr1dcn3-dataHRCV-BC--dataFFTCV-BC,dataHRCV-BCcn3-dataHRCV-BC--dataFFTCV-BC,fftDatacn3BC-dataHRCV-BC--dataFFTCV-BC,welchDatacn3BC-dataHRCV-BC--dataFFTCV-BC,dataHRcn3BC-dataHRCV-BC--dataFFTCV-BC,gaussianDatacn3-dataHRCV-BC--dataFFTCV-BC,gaussianDatacn3BC-dataHRCV-BC--dataFFTCV-BC,dataCorr1dcn3BC-dataHRCV-BC--dataFFTCV-BC,dataHRCV-BCcn3-fftDataBC--dataFFTCV-BC,fftDatacn3BC-fftDataBC--dataFFTCV-BC,welchDatacn3BC-fftDataBC--dataFFTCV-BC,dataHRCV-BCcn3-welchDataBC--dataFFTCV-BC,fftDatacn3BC-welchDataBC--dataFFTCV-BC,welchDatacn3BC-welchDataBC--dataFFTCV-BC,dataHRCV-BCcn3-dataHRBC--dataFFTCV-BC,fftDatacn3BC-dataHRBC--dataFFTCV-BC,welchDatacn3BC-dataHRBC--dataFFTCV-BC,dataHRCV-BCcn3-gaussianData--dataFFTCV-BC,fftDatacn3BC-gaussianData--dataFFTCV-BC,welchDatacn3BC-gaussianData--dataFFTCV-BC,dataHRCV-BCcn3-gaussianDataBC--dataFFTCV-BC,fftDatacn3BC-gaussianDataBC--dataFFTCV-BC,welchDatacn3BC-gaussianDataBC--dataFFTCV-BC,dataHRCV-BCcn3-dataCorr1dBC--dataFFTCV-BC,fftDatacn3BC-dataCorr1dBC--dataFFTCV-BC,welchDatacn3BC-dataCorr1dBC--dataFFTCV-BC,dataHRCV-BCcn3-fftDatacn3--dataFFTCV-BC,fftDatacn3BC-fftDatacn3--dataFFTCV-BC,welchDatacn3BC-fftDatacn3--dataFFTCV-BC,dataHRCV-BCcn3-welchDatacn3--dataFFTCV-BC,fftDatacn3BC-welchDatacn3--dataFFTCV-BC,welchDatacn3BC-welchDatacn3--dataFFTCV-BC,dataHRCV-BCcn3-dataHRcn3--dataFFTCV-BC,fftDatacn3BC-dataHRcn3--dataFFTCV-BC,welchDatacn3BC-dataHRcn3--dataFFTCV-BC,dataHRCV-BCcn3-dataCorr1dcn3--dataFFTCV-BC,fftDatacn3BC-dataCorr1dcn3--dataFFTCV-BC,welchDatacn3BC-dataCorr1dcn3--dataFFTCV-BC,fftDatacn3BC-dataHRCV-BCcn3--dataFFTCV-BC,welchDatacn3BC-dataHRCV-BCcn3--dataFFTCV-BC,dataHRcn3BC-dataHRCV-BCcn3--dataFFTCV-BC,gaussianDatacn3-dataHRCV-BCcn3--dataFFTCV-BC,gaussianDatacn3BC-dataHRCV-BCcn3--dataFFTCV-BC,dataCorr1dcn3BC-dataHRCV-BCcn3--dataFFTCV-BC,welchDatacn3BC-fftDatacn3BC--dataFFTCV-BC,dataHRcn3BC-fftDatacn3BC--dataFFTCV-BC,gaussianDatacn3-fftDatacn3BC--dataFFTCV-BC,gaussianDatacn3BC-fftDatacn3BC--dataFFTCV-BC,dataCorr1dcn3BC-fftDatacn3BC--dataFFTCV-BC,dataHRcn3BC-welchDatacn3BC--dataFFTCV-BC,gaussianDatacn3-welchDatacn3BC--dataFFTCV-BC,gaussianDatacn3BC-welchDatacn3BC--dataFFTCV-BC,dataCorr1dcn3BC-welchDatacn3BC--dataFFTCV-BC,welchDatacn3BC-dataHRCV-BC--dataWCV-BC,welchDatacn3BC-fftDataBC--dataWCV-BC,welchDatacn3BC-welchDataBC--dataWCV-BC,welchDatacn3BC-dataHRBC--dataWCV-BC,welchDatacn3BC-gaussianData--dataWCV-BC,welchDatacn3BC-gaussianDataBC--dataWCV-BC,welchDatacn3BC-dataCorr1dBC--dataWCV-BC,welchDatacn3BC-fftDatacn3--dataWCV-BC,welchDatacn3BC-welchDatacn3--dataWCV-BC,welchDatacn3BC-dataHRcn3--dataWCV-BC,welchDatacn3BC-dataCorr1dcn3--dataWCV-BC,welchDatacn3BC

In [438]:
print("Top ten highest accuracies")

topTen.reverse()
for top in topTen:
    print(
            f"Accuracy {round(top[0],2)} : subject {top[1]}, feature:{top[2]}, kernel{top[3]}, C:{top[4]}"
        )


Top ten highest accuracies
Accuracy 0.99 : subject 8, feature:welchDatacn3BC-dataWCV-BC--dataFFTCV-BC, kernelrbf, C:2.5
Accuracy 0.99 : subject 8, feature:dataHRCV-BCcn3-dataWCV-BC--dataFFTCV-BC, kernelrbf, C:2.5
Accuracy 0.99 : subject 8, feature:welchDatacn3BC-dataHRCV-BC--dataFFTCV-BC, kernelrbf, C:2.5
Accuracy 0.99 : subject 8, feature:gaussianDataBC-dataHRCV-BC--dataFFTCV-BC, kernelrbf, C:2.5
Accuracy 0.98 : subject 5, feature:welchDatacn3BC-dataHRCV-BC--dataWCV-BC, kernelrbf, C:2.5
Accuracy 0.98 : subject 5, feature:dataHRCV-BCcn3-dataWCV-BC--dataFFTCV-BC, kernelrbf, C:2.5
Accuracy 0.97 : subject 5, feature:welchDatacn3BC--dataWCV-BC, kernelrbf, C:2.5
Accuracy 0.97 : subject 5, feature:welchDatacn3BC-dataHRCV-BC--dataFFTCV-BC, kernelrbf, C:2.5
Accuracy 0.97 : subject 8, feature:dataHRCV-BCcn3-dataHRCV-BC--dataFFTCV-BC, kernelrbf, C:2.5
Accuracy 0.97 : subject 5, feature:dataHRCV-BCcn3-dataHRCV-BC--dataFFTCV-BC, kernelrbf, C:2.5


In [439]:
print("Top ten highest accuracies")
#del(topThreeThisSubject[0])
#len(topThreeAllSubjects[0])
for subTop in topThreeAllSubjects:
    subTop.reverse()
topThreeAllSubjects.sort(reverse=True)
for subTop in topThreeAllSubjects:
    print("")
    #subTop.reverse()
    for top in subTop:
        print(
                f"Accuracy {round(top[0],2)} : subject {top[1]}, feature:{top[2]}, kernel{top[3]}, C:{top[4]}"
            )

Top ten highest accuracies

Accuracy 0.99 : subject 8, feature:welchDatacn3BC-dataWCV-BC--dataFFTCV-BC, kernelrbf, C:2.5
Accuracy 0.99 : subject 8, feature:dataHRCV-BCcn3-dataWCV-BC--dataFFTCV-BC, kernelrbf, C:2.5

Accuracy 0.98 : subject 5, feature:welchDatacn3BC-dataHRCV-BC--dataWCV-BC, kernelrbf, C:2.5
Accuracy 0.98 : subject 5, feature:dataHRCV-BCcn3-dataWCV-BC--dataFFTCV-BC, kernelrbf, C:2.5

Accuracy 0.96 : subject 7, feature:fftDatacn3BC-dataWCV-BC--dataFFTCV-BC, kernelrbf, C:2.5
Accuracy 0.96 : subject 7, feature:fftDatacn3BC--dataFFTCV-BC, kernelrbf, C:2.5

Accuracy 0.95 : subject 4, feature:welchDatacn3BC-dataWCV-BC--dataFFTCV-BC, kernelrbf, C:2.5
Accuracy 0.95 : subject 4, feature:welchDatacn3BC--dataFFTCV-BC, kernelrbf, C:2.5

Accuracy 0.92 : subject 2, feature:welchDatacn3BC-dataWCV-BC--dataFFTCV-BC, kernelrbf, C:2.5
Accuracy 0.91 : subject 2, feature:fftDatacn3BC-dataWCV-BC--dataFFTCV-BC, kernelrbf, C:2.5

Accuracy 0.91 : subject 1, feature:welchDatacn3BC-dataHRCV-BC--dat

In [440]:
subjAv = np.zeros(subjectResultArray.shape[0])
for s, subj in enumerate(subjectResultArray):
    subjAv[s] = 0
    if all(subj == None):
        continue
    subjAv[s] = np.mean(subj[np.where(subj != None)])
print(subjAv)
pd.set_option("display.max_columns", None)
table = pd.DataFrame(np.array(comboAverage), columns=headers)
display(table)

print(
    f"Max average accuracy {np.max(np.array(comboAverage[np.where(np.isnan(comboAverage)==False)]))}"
)
maxVal = np.max(np.array(comboAverage[np.where(np.isnan(comboAverage) == False)]))
maxInd = int(np.where(comboAverage == maxVal)[1][0])
# print(headers[maxInd])
# print(nrOfSubjects[maxInd])
# print(list(nameDict.keys())[combinationList[maxInd][0]])
# print(list(kernelDict.keys())[combinationList[maxInd][1]])
# print(f"C: {combinationList[maxInd][2]}")


[0.         0.80299383 0.67734396 0.51103109 0.79657579 0.82726337
 0.59042088 0.78298225 0.83038194 0.54841821]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107
0,0.81992,0.811388,0.835082,0.853131,0.84564,0.745757,0.736982,0.737041,0.743736,0.455042,0.750755,0.736608,0.746391,0.750096,0.754149,0.713276,0.68798,0.705638,0.710322,0.476287,0.709891,0.714628,0.717571,0.721569,0.489887,0.826783,0.828911,0.836499,0.871474,0.610997,0.678088,0.78641,0.746762,0.814086,0.784428,0.738565,0.750899,0.706362,0.711578,0.790895,0.811847,0.847593,0.72014,0.747781,0.792374,0.759835,0.571174,0.580804,0.578197,0.648802,0.64082,0.63178,0.76607,0.765448,0.771495,0.746305,0.778833,0.781659,0.781817,0.813208,0.817318,0.765048,0.768874,0.775585,0.7332,0.747303,0.749066,0.735816,0.745728,0.740209,0.68183,0.685333,0.681341,0.709229,0.71991,0.718624,0.791442,0.822784,0.687382,0.746832,0.769008,0.740536,0.814642,0.67911,0.78131,0.791457,0.749388,0.68664,0.782298,0.803803,0.759938,0.859981,0.412682,0.635319,0.489735,0.476069,0.480328,0.468246,0.500206,0.736021,0.479398,0.479922,0.816419,0.505264,0.477906,0.476918,0.474939,0.471115


Max average accuracy 0.8714739213118844


In [441]:
#np.argpartition(comboAverage, -4)
#print(headers[comboAverage.argsort()[0][-5:][::-1]])
top5 = headers[comboAverage.argsort()[0][-5:][::-1]]
top5Vals = comboAverage[0][comboAverage.argsort()[0][-5:][::-1]]
for maxInd,res in zip(top5, top5Vals):
    print(list(nameDict.keys())[combinationList[maxInd][0]])
    print(list(kernelDict.keys())[combinationList[maxInd][1]])
    print(res)
    print()

welchDatacn3BC-dataWCV-BC--dataFFTCV-BC
rbf
0.8714739213118844

welchDatacn3BC-dataHRCV-BC--dataWCV-BC
rbf
0.8599810606060605

welchDatacn3BC--dataFFTCV-BC
rbf
0.8531306116722784

welchDatacn3BC-dataHRCV-BC--dataFFTCV-BC
rbf
0.8475925146527

welchDatacn3BC--dataWCV-BC
rbf
0.8456396526998379



In [442]:
#np.argpartition(comboAverage, -4)
#print(headers[comboAverage.argsort()[0][-5:][::-1]])
top5 = headers[comboTotal.argsort()[0][-5:][::-1]]
top5Vals = comboTotal[0][comboTotal.argsort()[0][-5:][::-1]] /9
for maxInd,res in zip(top5, top5Vals):
    print(list(nameDict.keys())[combinationList[maxInd][0]])
    print(list(kernelDict.keys())[combinationList[maxInd][1]])
    print(res)
    print()

welchDatacn3BC-dataWCV-BC--dataFFTCV-BC
rbf
0.8714739213118844

welchDatacn3BC-dataHRCV-BC--dataWCV-BC
rbf
0.8599810606060605

welchDatacn3BC--dataFFTCV-BC
rbf
0.8531306116722784

welchDatacn3BC-dataHRCV-BC--dataFFTCV-BC
rbf
0.8475925146527

welchDatacn3BC--dataWCV-BC
rbf
0.8456396526998379



In [443]:
table = pd.DataFrame(np.array(comboTotal), columns=headers)
display(table)

maxVal2 = np.max(np.array(comboTotal[np.where(np.isnan(comboTotal) == False)]))

for maxInd2 in np.where(comboTotal == maxVal2)[1]:

    # maxInd2 = int(np.where(comboTotal == maxVal2)[1])
    print(headers[maxInd2])
    print(nrOfSubjects[maxInd2])
    print(
        f"Max total significant accuracy then averaged {maxVal2/nrOfSubjects[maxInd2]} "
    )

    print(list(nameDict.keys())[combinationList[maxInd2][0]])
    print(list(kernelDict.keys())[combinationList[maxInd2][1]])
    print(f"C: {combinationList[maxInd2][2]} ")


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107
0,7.37928,7.302491,7.515741,7.678176,7.610757,6.711817,6.632834,6.633368,6.69362,4.095381,6.756799,6.629473,6.717517,6.750863,6.787338,6.419485,6.191823,6.350746,6.392894,4.286581,6.389021,6.431649,6.458143,6.494118,4.408982,7.441043,7.460203,7.528495,7.843265,5.498971,6.102791,7.077691,6.720861,7.32677,7.059852,6.647081,6.758088,6.357262,6.4042,7.118056,7.306626,7.628333,6.481263,6.730032,7.131368,6.838512,5.140564,5.227238,5.203774,5.839217,5.767381,5.686021,6.894633,6.889028,6.943456,6.716743,7.009493,7.034934,7.036349,7.318868,7.355861,6.885429,6.919862,6.980269,6.598796,6.725725,6.741592,6.622345,6.711552,6.661882,6.136467,6.167996,6.132067,6.383058,6.479191,6.467612,7.122981,7.405059,6.186437,6.721488,6.92107,6.664822,7.331779,6.111991,7.031792,7.12311,6.744496,6.179759,7.040686,7.234228,6.839444,7.73983,3.714134,5.717872,4.407618,4.284617,4.322948,4.214215,4.501855,6.624192,4.314583,4.319297,7.347769,4.547373,4.301157,4.292263,4.274453,4.240039


28
9.0
Max total significant accuracy then averaged 0.8714739213118844 
welchDatacn3BC-dataWCV-BC--dataFFTCV-BC
rbf
C: 2.5 


In [444]:
# maxAcc = np.max([res for res in np.array(
#     subjectResultArray[subjectResultArray != None])])
# print(f"Max single accuracy position {np.where(subjectResultArray == maxAcc)}")
# print(np.max([res for res in np.array(
#     subjectResultArray[subjectResultArray != None])]))
